In [1]:
import os 
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import utils 
from tensorflow.keras import layers
from tensorflow.keras import models



In [2]:
with open('../../data/data_model/data_dict.pkl','rb') as f:
    data = pickle.load(f) 

In [3]:
mapping_label =['Bronchiectasis','Bronchiolitis', 'LRTI', 'Pneumonia','URTI','Asthma']

X=data['features']
y = pd.Series(data['label'])
y = y.replace(mapping_label,'respiratory_infection')
y = y.values

le=LabelEncoder()
y=le.fit_transform(y)

In [4]:
X = np.reshape(X, (*X.shape,1))

In [5]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.4, stratify=y)

In [7]:
for i in range(3):
    print('Train',i,np.sum(ytrain == i))
    print('Test',i,np.sum(ytest == i))

Train 0 476
Test 0 317
Train 1 21
Test 1 14
Train 2 55
Test 2 37


<h1> Deep Learning Model<h1>

In [8]:
input_shape = X.shape[1:]
n_classes= len(le.classes_)
print(input_shape,n_classes)

(77, 674, 1) 3


In [9]:

filter_size = 2
# Construct model 
model = models.Sequential()
model.add(layers.Conv2D(filters=16, kernel_size=filter_size,
                 input_shape=input_shape, activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(filters=32, kernel_size=filter_size, activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(filters=64, kernel_size=filter_size, activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(filters=128, kernel_size=filter_size, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(1024))
model.add(layers.Dense(512))
model.add(layers.Dense(64))
model.add(layers.Dense(3, activation='softmax')) 

In [ ]:
model.summary()


In [ ]:
from keras import backend as K

callback = tf.keras.callbacks.EarlyStopping(patience=3, 
                          verbose=1, 
                          min_delta=0.001, 
                          monitor='val_loss', 
                          mode='auto', 
                          restore_best_weights=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
K.set_value(model.optimizer.learning_rate, 0.001)

history = model.fit(Xtrain,
          ytrain,
          epochs=100,
          batch_size=128,
          validation_data=(Xtest, ytest))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label = 'training acc')
plt.plot(history.history['val_accuracy'], label = 'validation acc')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label = 'training loss')
plt.plot(history.history['val_loss'], label = 'validation loss')
plt.legend()
plt.title('Loss')

In [ ]:
print("Evaluate model on test data")
results = model.evaluate(Xtest, ytest)
print("test loss, test acc:", results)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_predict = model.predict(Xtest)
y_predict = np.argmax(y_predict,axis=1)
confusion_matrix(ytest, y_predict)

In [ ]:
print(classification_report(ytest, y_predict, target_names=le.classes_))